In [32]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold

X = pd.read_csv(r"../data/df_X.csv", engine="pyarrow")
y= pd.read_csv(r"../data/df_y.csv", engine="pyarrow")

# Código do Decision Tree

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = DecisionTreeClassifier(random_state=42)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

melhor_modelo = None
melhor_score = 0

for train_index, val_index in kf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_tr, y_tr)
    
    score = accuracy_score(y_val, model.predict(X_val))
    
    if score > melhor_score:
        melhor_score = score
        melhor_modelo = model  # salva o melhor modelo

print(f"Melhor acurácia entre os folds: {melhor_score:.4f}")

Melhor acurácia entre os folds: 0.6694


In [ ]:
y_pred = melhor_modelo.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)   
rec = recall_score(y_test, y_pred)      

print(f"Acurácia : {acc:.4f}")
print(f"Precisão : {prec:.4f}")
print(f"Recall   : {rec:.4f}")

Acurácia : 0.6250
Precisão : 0.3235
Recall   : 0.2444
